In [ ]:
#The Movies Dataset Cleanup Utility and Book Summaries Fuzzy Matcher
#@author Velizar Petrov

from rapidfuzz import process, fuzz
import pandas as pd
import numpy as np
import ast


# 1. Load datasets
movies = pd.read_csv("Datasets\\The Movies Dataset\\movies_metadata.csv", low_memory=False)
ratings = pd.read_csv("Datasets\\The Movies Dataset\\ratings.csv")
keywords = pd.read_csv("Datasets\\The Movies Dataset\\keywords.csv")
credits = pd.read_csv("Datasets\\The Movies Dataset\\credits.csv")


# 2. Clean movies_metadata
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
movies = movies.dropna(subset=['id'])
movies['id'] = movies['id'].astype(int)

# Drop duplicate IDs and titles
movies = movies.drop_duplicates(subset='id')
movies = movies.drop_duplicates(subset=['title', 'release_date'])


# 3. Clean ratings
ratings['movieId'] = pd.to_numeric(ratings['movieId'], errors='coerce')
ratings = ratings.dropna(subset=['movieId'])
ratings['movieId'] = ratings['movieId'].astype(int)

# Drop duplicate (userId, movieId)
ratings = ratings.drop_duplicates(subset=['userId', 'movieId'])

# Align with movies
ratings = ratings[ratings['movieId'].isin(movies['id'])]


# 4. Clean and parse keywords
keywords['id'] = pd.to_numeric(keywords['id'], errors='coerce')
keywords = keywords.dropna(subset=['id'])
keywords['id'] = keywords['id'].astype(int)
keywords = keywords.drop_duplicates(subset='id')

# Parse keyword list
def parse_keywords(x):
    try:
        kws = ast.literal_eval(x)
        return [d['name'] for d in kws] if isinstance(kws, list) else []
    except:
        return []

keywords['keywords'] = keywords['keywords'].apply(parse_keywords)


# 5. Clean and parse credits

credits['id'] = pd.to_numeric(credits['id'], errors='coerce')
credits = credits.dropna(subset=['id'])
credits['id'] = credits['id'].astype(int)
credits = credits.drop_duplicates(subset='id')

# Parse JSON-like strings
def parse_cast(x, top_n=3):
    try:
        cast = ast.literal_eval(x)
        return [d['name'] for d in cast[:top_n]] if isinstance(cast, list) else []
    except:
        return []

def parse_director(x):
    try:
        crew = ast.literal_eval(x)
        directors = [d['name'] for d in crew if d.get('job') == 'Director']
        return directors[:1] if directors else []
    except:
        return []

credits['cast'] = credits['cast'].apply(parse_cast)
credits['director'] = credits['crew'].apply(parse_director)
credits = credits.drop(columns=['crew'])  # drop raw crew

# 6. Merge all content features

# Keep genres from movies
def parse_genres(x):
    try:
        g = ast.literal_eval(x)
        return [d['name'] for d in g] if isinstance(g, list) else []
    except:
        return []

movies['genres'] = movies['genres'].apply(parse_genres)

# Merge with keywords and credits
content = movies[['id', 'title', 'genres']].merge(
    keywords[['id', 'keywords']], on='id', how='left'
).merge(
    credits[['id', 'cast', 'director']], on='id', how='left'
)

# Fill missing lists
for col in ['genres', 'keywords', 'cast', 'director']:
    content[col] = content[col].apply(lambda x: x if isinstance(x, list) else [])

# Build combined feature string
content['content_features'] = content['genres'] + content['keywords'] + content['cast'] + content['director']
content['content_features'] = content['content_features'].apply(lambda x: " ".join(x))


# 7. Save cleaned datasets
movies.to_csv("movies_clean.csv", index=False)
ratings.to_csv("ratings_clean.csv", index=False)
content[['id', 'title', 'content_features']].to_csv("content_features.csv", index=False)

print("  Cleaning complete! Saved:")
print("- movies_clean.csv")
print("- ratings_clean.csv")
print("- content_features.csv")

In [ ]:
# Load CSV files
goodbooks = pd.read_csv("goodbooks_10k_rating_and_description.csv")
summaries = pd.read_csv("booksummaries.csv")

# Normalize titles
goodbooks['book_title_clean'] = goodbooks['book_title'].str.strip().str.lower()
summaries['title_clean'] = summaries['title'].str.strip().str.lower()

# Create lookup dictionary
desc_dict = summaries.set_index('title_clean')['description'].to_dict()

# Reporting variables
initial_missing = goodbooks['book_desc'].isna().sum()
already_filled = goodbooks['book_desc'].notna().sum()

# Exact match pass 
exact_matches = 0
def exact_match(row):
    global exact_matches
    if pd.isna(row['book_desc']):
        desc = desc_dict.get(row['book_title_clean'])
        if desc:
            exact_matches += 1
            return desc
    return row['book_desc']

goodbooks['book_desc'] = goodbooks.apply(exact_match, axis=1)

# Fuzzy match pass 
fuzzy_matches = 0
title_list = summaries['title_clean'].tolist()
missing_mask = goodbooks['book_desc'].isna()
title_to_desc = desc_dict

def fuzzy_fill(title):
    global fuzzy_matches
    match = process.extractOne(title, title_list, scorer=fuzz.token_sort_ratio, score_cutoff=90)
    if match:
        fuzzy_matches += 1
        return title_to_desc.get(match[0])
    return None

goodbooks.loc[missing_mask, 'book_desc'] = goodbooks.loc[missing_mask, 'book_title_clean'].apply(fuzzy_fill)

# Final reporting 
final_missing = goodbooks['book_desc'].isna().sum()
final_filled = goodbooks['book_desc'].notna().sum()

print("=== Matching Report ===")
print(f"Total books: {len(goodbooks)}")
print(f"Descriptions initially filled: {already_filled}")
print(f"Filled via exact title match: {exact_matches}")
print(f"Filled via fuzzy title match: {fuzzy_matches}")
print(f"Still missing: {final_missing}")

# Save result
goodbooks.drop(columns=['book_title_clean'], inplace=True)
goodbooks.to_csv("goodbooks_10k_rating_and_description_fuzzy_filled.csv", index=False)
